In [ ]:
import wandb
import pandas as pd
from fastai.vision.all import *
from fastai.callback.wandb import WandbCallback
import params
from utils import get_predictions, create_iou_table, MIOU, BackgroundIOU, \
                  RoadIOU, TrafficLightIOU, TrafficSignIOU, PersonIOU, VehicleIOU, BicycleIOU

Creando configuración para pasarle a W&B para controlar los hiperametros en el entrenamiento. 

In [ ]:

train_config = SimpleNamespace(
    framework="fastai",
    img_size=(180, 320),
    batch_size=8,
    augment=True, # use data augmentation
    epochs=10, 
    lr=2e-3,
    pretrained=True,  # whether to use pretrained encoder
    seed=42,
)

¿Qué es fastai? Es una librería de alto nivel que usa PyTorch por debajo. Es decir, es una librería que nos permite entrenar modelos de deep learning de forma sencilla y rápida. En este caso, usaremos la parte de visión de fastai, que nos permite entrenar modelos de clasificación de imágenes.

Para en entrenamiento necesitamos:
- Un modelo
- Un optimizador (lr)
- Especificar tamaño de las imágenes
- Un criterio de evaluación que lo haremos exportando las métricas desde utils.py (iou)
- Batch size

In [ ]:
# fijar la semilla para reproducibilidad
set_seed(train_config.seed, reproducible=True)

In [ ]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="training", config=train_config)

In [ ]:
processed_data_at = run.use_artifact(f'{params.PROCESSED_DATA_AT}:latest')
processed_dataset_dir = Path(processed_data_at.download())
df = pd.read_csv(processed_dataset_dir / 'data_split.csv')

In [ ]:
df.head(15)

In [ ]:
df = df[df.Stage != 'test'].reset_index(drop=True)
df['is_valid'] = df.Stage == 'valid'


In [ ]:
df.sample(5)

In [ ]:
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

In [ ]:
# assign paths
df["image_fname"] = [processed_dataset_dir/f'images/{f}' for f in df.File_Name.values]
df["label_fname"] = [label_func(f) for f in df.image_fname.values]

In [ ]:
df.sample(5)

In [ ]:
def get_data(df, bs=4, img_size=(180, 320), augment=True):
    """Create dataloaders from dataframe
    Input:
        df: dataframe with columns image_fname, label_fname, is_valid
        bs: batch size
        img_size: image size
        augment: whether to use data augmentation
    Output:
        dataloaders"""
    block = DataBlock(blocks=(ImageBlock, MaskBlock(codes=params.BDD_CLASSES)),
                  get_x=ColReader("image_fname"),
                  get_y=ColReader("label_fname"),
                  splitter=ColSplitter(),
                  item_tfms=Resize(img_size),
                  batch_tfms=aug_transforms() if augment else None,
                 )
    return block.dataloaders(df, bs=bs)

In [ ]:
config = wandb.config
config

In [ ]:
dls = get_data(df, bs=config.batch_size, img_size=config.img_size, augment=config.augment)

Usaremos la intersección sobre métricas de unión: media en todas las clases (MIOU) y pagaré para cada clase por separado. Nuestro modelo será un unet basado en la columna vertebral resnet18 previamente entrenada.

In [ ]:
metrics = [MIOU(), BackgroundIOU(), RoadIOU(), TrafficLightIOU(), \
           TrafficSignIOU(), PersonIOU(), VehicleIOU(), BicycleIOU()]

learn = unet_learner(dls, arch=resnet18, pretrained=config.pretrained, metrics=metrics)


En fastai, ya tenemos un callback que se integra estrechamente con W&B (WandB, que se refiere a "Weights and Biases", una plataforma de seguimiento y visualización de experimentos). Solo necesitamos pasar el WandbCallback al objeto learner, y estamos listos para empezar. El callback registrará automáticamente todas las variables útiles para nosotros. Por ejemplo, cualquier métrica que pasemos al objeto learner será rastreada por el callback.



En el contexto de fastai, un "callback" es una función o clase que se utiliza para personalizar o extender el comportamiento de un objeto learner durante el entrenamiento de un modelo de aprendizaje automático. Creamos un callback llamado "WandbCallback" que se ha diseñado específicamente para trabajar con W&B. Para utilizar este callback, simplemente necesitamos agregarlo al objeto learner que estamos utilizando para entrenar el modelo. El callback se encargará automáticamente de registrar y enviar a W&B todas las métricas y variables relevantes.


In [ ]:
callbacks = [
    SaveModelCallback(monitor='miou'),
    WandbCallback(log_preds=False, log_model=True)
]
     

In [ ]:
learn.fit_one_cycle(config.epochs, config.lr, cbs=callbacks)

In [ ]:
samples, outputs, predictions = get_predictions(learn)
table = create_iou_table(samples, outputs, predictions, params.BDD_CLASSES)
wandb.log({"pred_table":table})

In [ ]:
scores = learn.validate()
metric_names = ['final_loss'] + [f'final_{x.name}' for x in metrics]
final_results = {metric_names[i] : scores[i] for i in range(len(scores))}
for k,v in final_results.items(): 
    wandb.summary[k] = v

In [ ]:
wandb.finish()

In [ ]:

## pruebita para probar torch 
# import sys
# import platform
# import torch
# import pandas as pd
# import sklearn as sk

# has_gpu = torch.cuda.is_available()
# has_mps = getattr(torch,'has_mps',False)
# device = "mps" if getattr(torch,'has_mps',False) \
#     else "gpu" if torch.cuda.is_available() else "cpu"

# print(f"Python Platform: {platform.platform()}")
# print(f"PyTorch Version: {torch.__version__}")
# print()
# print(f"Python {sys.version}")
# print(f"Pandas {pd.__version__}")
# print(f"Scikit-Learn {sk.__version__}")
# print("GPU is", "available" if has_gpu else "NOT AVAILABLE")
# print("MPS (Apple Metal) is", "AVAILABLE" if has_mps else "NOT AVAILABLE")
# print(f"Target device is {device}")